In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
plt.style.use("seaborn-poster")
from combined_phot_funcs import *
from hexLevelAnalyses import get_sigRats_fromMeanList
from photometryQuantifications import *
%matplotlib qt

### load trial-level dataframe

In [2]:
loadpath = "/Volumes/Tim/Photometry/10MfRatDataSet/"
df = pd.read_csv(loadpath+"triframe.csv")
photrats = PhotRats(None)
photrats.triframe = df.copy()
del df
photrats.directory_prefix = loadpath

### create new dataframes with relevant columns for regression, and plot results

- create and save same vs alt path regression df
- create df for visualization of choice probability as a function of reward on same vs previous path-to-goal

In [4]:
def get_log_pchoos_v_costNben(photrats):
    df = photrats.triframe.copy()
    df['ratcodes'] = df.rat.astype('category').cat.codes
    seshs=df.session.unique()
    for s in range(len(seshs)):
        sdf = df.loc[(df.session==seshs[s])].copy()
        rdf = pd.DataFrame({'pRwdDif':photrats.get_lr_dif_val(\
            sdf,'nom_rwd'),'ldif':photrats.get_lr_dif_val(sdf,'dist')})
        rdf['choose_L'] = sdf.lrchoice.values
        rdf['session']=s
        rdf['rat'] = sdf.ratcodes.values
        rdf['tri'] = sdf.tri.values
        rdf['block'] = sdf.block.values
        rdf.loc[:,"rt-1"] = np.nan
        sdf.reset_index(inplace=True)
        for p in range(3):
            lagInds = sdf.loc[sdf.port==p,:].index
            rdf.loc[lagInds,"rt-1"] = sdf.loc[lagInds,'rwd'].shift(1).values
        if s == 0:
            photrats.regdf = rdf
        else:
            photrats.regdf = photrats.regdf.append(rdf,ignore_index=True)
    photrats.regdf.loc[photrats.regdf.choose_L==2,'choose_L']=np.nan

- need to run an analysis analagous to DA pre-CP same vs alt comparison. Need to reduce analysis to choices where alt or same was taken previously... 

In [5]:
def add_prevRwdTakenAndAlt(photrats):
    '''add columns indicating whether rat was rewarded
    when previously taking the same (taken) or alternative path
    to the currently chosen port.'''
    photrats.triframe.loc[:,["prev_rwd_taken","prev_rwd_alt"]] = np.nan
    photrats.triframe.loc[:,["prev_rwd_taken","prev_rwd_alt"]] = -100
    photrats.triframe.loc[(photrats.triframe["samePath_t-1"]==0),"prev_rwd_alt"] = \
        photrats.triframe.loc[(photrats.triframe["samePath_t-1"]==0),"rwd"].values
    photrats.triframe.loc[(photrats.triframe["samePath_t-1"]==1),"prev_rwd_taken"] = \
        photrats.triframe.loc[(photrats.triframe["samePath_t-1"]==1),"rwd"].values
    photrats.triframe.loc[photrats.triframe.prev_rwd_alt==-100,"prev_rwd_alt"] = np.nan
    photrats.triframe.loc[photrats.triframe.prev_rwd_taken==-100,"prev_rwd_taken"] = np.nan

In [6]:
def get_back2samePortTrials(photrats):
    photrats.triframe.loc[:,"same2"] = 0
    photrats.triframe.loc[photrats.triframe.port==photrats.triframe.fromport2,"same2"] = 1
    
def get_portHist(photrats):
    photrats.triframe.loc[:,"fromport"] = photrats.triframe.loc[:,"port"].shift(1).values
    photrats.triframe.loc[:,"fromport2"] = photrats.triframe.loc[:,"port"].shift(2).values
    
def add_rewardLagColumns2triframe(photrats,lags=1):
    for l in range(1,lags+1):
        photrats.triframe.loc[:,"rt-"+str(l)] = np.nan
        photrats.triframe.loc[:,"samePath_t-"+str(l)] = 0
        for p in range(3):
            lagInds = photrats.triframe.loc[photrats.triframe.port==p,:].index
            photrats.triframe.loc[lagInds,"rt-"+str(l)] = photrats.triframe.loc[lagInds,'rwd'].shift(l).values
            photrats.triframe.loc[lagInds,"samePath_t-"+str(l)] = (photrats.triframe.loc[lagInds,\
                'fromport']==photrats.triframe.loc[lagInds,'fromport'].shift(l)).astype("int8")

In [7]:
get_portHist(photrats)
get_back2samePortTrials(photrats)
add_rewardLagColumns2triframe(photrats)
#add_prevRwdTakenAndAlt(photrats)

In [15]:
photrats.triframe.tail()

,Unnamed: 0,index,Unnamed: 0.1,port,rwd,nom_rwd_a,nom_rwd_b,tri,block,nom_rwd_c,...,Q_a,Q_b,Q_c,Q_chosen,lrchoice,fromport,fromport2,same2,rt-1,samePath_t-1
15996,15996,109605725,109605730,0,1.0,90.0,50.0,59,3,10.0,...,0.7505,0.839,0.002620,0.75050,1,1.0,2.0,0,1.0,1
15997,15997,109619971,109619976,2,0.0,90.0,50.0,60,3,10.0,...,0.8896,0.839,0.002620,0.00262,1,0.0,1.0,0,0.0,0
15998,15998,109626772,109626777,1,0.0,90.0,50.0,61,3,10.0,...,0.8896,0.839,0.001156,0.83900,1,2.0,0.0,0,1.0,1
15999,15999,109667620,109667625,0,0.0,90.0,50.0,62,3,10.0,...,0.8896,0.370,0.001156,0.88960,1,1.0,2.0,0,1.0,1
16000,16000,109671825,109671830,1,0.0,50.0,10.0,0,4,90.0,...,0.3926,0.370,0.001156,0.37000,0,0.0,1.0,1,0.0,0


# should be an easy way to add "stay" column to df, instead of creating another dataframe for wsls.
- this is actually equivalent to "same2"
- need a way to compare same2==0 and same2==1.
- take all data where samePath_t-1 == 1 OR samePath_t-1 == 0 (for taken vs alt)
- regress same2 (logistic) to rt-1 through rt-5, and add in lrchoice for leftward bias. intercept will capture return bias

# actually, can't run any type of wsls analysis here. Really important to control for same2, so should reframe as a choose L analysis. that way it will be similar to the earlier choice analysis as well.
- lrchoice ~ 1 + Lrt.1 + Lrt.2 + ... Lrt.5, data = tridf[tridf.same2==1 & tridf.samePath_t-1_left==0/1,]
- need to calculate reward history of left port five rewards back
  - might have to first mark reward history of each port... rt-1_a, rt-1_b, ... then use a function to grab the left, similart to get_lr_dif_val() "get_left_val()"
- need to calculate samePath_t-1 to left port one trial back.
  - last time rat ran to left port (port==X) did rat start at current starting port?
- also need to get same2 for left port
  - this will be where left was chosen last.. not ABA but Left-current-Unknown. Can also do this for wsls... just looking where port of interest was chosen before current trial

In [8]:
def add_rewardLagColumns2triframeByPort(photrats,lags=5):
    portStrings = ["a","b","c"]
    for l in range(1,lags+1):
        photrats.triframe.loc[:,["rt-"+str(l)+"_"+s for s in portStrings]] = np.nan
        for p in range(3):
            lagInds = photrats.triframe.loc[photrats.triframe.port==p,:].index
            photrats.triframe.loc[lagInds,"rt-"+str(l)+"_"+portStrings[p]] = photrats.triframe.loc[lagInds,'rwd'].shift(l).values
        for sesh in photrats.triframe.session.unique():
            photrats.triframe.loc[photrats.triframe.session==sesh,["rt-"+str(l)+"_"+s for s in portStrings]] = \
            photrats.triframe.loc[photrats.triframe.session==sesh,["rt-"+str(l)+"_"+s for s in portStrings]].fillna(method='ffill')
            
def add_samePathLagColumns2triframeByPort(photrats,lags=1):
    portStrings = ["a","b","c"]
    for l in range(1,lags+1):
        photrats.triframe.loc[:,["samePath_t-"+str(l)+"_"+s for s in portStrings]] = np.nan
        for p in range(3):
            lagInds = photrats.triframe.loc[photrats.triframe.port==p,:].index
            photrats.triframe.loc[lagInds,"samePath_t-"+str(l)+"_"+portStrings[p]] = (photrats.triframe.loc[lagInds,\
                'fromport']==photrats.triframe.loc[lagInds,'fromport'].shift(l)).astype("int8")
        for sesh in photrats.triframe.session.unique():
            photrats.triframe.loc[photrats.triframe.session==sesh,["samePath_t-"+str(l)+"_"+s for s in portStrings]] = \
            photrats.triframe.loc[photrats.triframe.session==sesh,["samePath_t-"+str(l)+"_"+s for s in portStrings]].fillna(method='ffill')
            
def add_leftLastColumn(photrats):
    '''adds column indicating the left port was visited last
    (two port entries prior to choice of interest). Make sure
    triframe index has been reset'''
    photrats.triframe.loc[:,"leftLast"] = np.nan
    # for each trial, identify left port, and ask whether prior entry == left
    leftChoices = [2,0,1]
    for sesh in photrats.triframe.session.unique():
        dat = photrats.triframe.loc[photrats.triframe.session==sesh,:]
        for i in dat.index[2:]:
            p = int(dat.loc[:,"port"].shift(1)[i])
            if dat.loc[:,"port"].shift(2)[i]==leftChoices[p]:
                photrats.triframe.loc[i,"leftLast"] = 1
            else:
                photrats.triframe.loc[i,"leftLast"] = 0

In [9]:
photrats.triframe.reset_index(inplace=True)

In [10]:
dat = photrats.triframe.loc[photrats.triframe.session==3,:]

In [12]:
photrats.triframe.columns

Index(['Unnamed: 0', 'port', 'rwd', 'nom_rwd_a', 'nom_rwd_b', 'tri', 'block',
       'nom_rwd_c', 'lenAC', 'lenBC', 'lenAB', 'dtop', 'fiberloc',
       'session_type', 'session', 'rat', 'date', 'nom_rwd_chosen', 'acc',
       'lrchoice', 'fromport', 'fromport2', 'same2', 'rt-1', 'samePath_t-1',
       'leftLast', 'rt-1_a', 'rt-1_b', 'rt-1_c', 'rt-2_a', 'rt-2_b', 'rt-2_c',
       'rt-3_a', 'rt-3_b', 'rt-3_c', 'rt-4_a', 'rt-4_b', 'rt-4_c', 'rt-5_a',
       'rt-5_b', 'rt-5_c', 'samePath_t-1_a', 'samePath_t-1_b',
       'samePath_t-1_c'],
      dtype='object')

In [11]:
add_rewardLagColumns2triframeByPort(photrats,lags=5)
add_samePathLagColumns2triframeByPort(photrats,lags=1)
add_leftLastColumn(photrats)

In [73]:
photrats.directory_prefix+"triframe4sameValtLeftChoiceReg.csv"

'/Volumes/Tim/Photometry/10MfRatDataSet/triframe4sameValtLeftChoiceReg.csv'

In [12]:
for l in range(1,6):
    photrats.triframe.loc[:,"rt-"+str(l)+"_left"] = photrats.get_left_val(photrats.triframe,factor="rt-"+str(l))
photrats.triframe.loc[:,"samePath_t-1_left"] = photrats.get_left_val(photrats.triframe,factor="samePath_t-1")


In [12]:
photrats.triframe.loc[(photrats.triframe.leftLast==1)&(photrats.triframe["samePath_t-1_left"]==0),].shape[0]

3232

In [29]:
photrats.triframe.to_csv(photrats.directory_prefix+"triframe4sameValtLeftChoiceReg.csv")

- get values of left port

In [46]:
photrats.triframe.columns

Index(['index', 'Unnamed: 0', 'port', 'rwd', 'nom_rwd_a', 'nom_rwd_b', 'tri',
       'block', 'nom_rwd_c', 'lenAC', 'lenBC', 'lenAB', 'dtop', 'fiberloc',
       'session_type', 'session', 'rat', 'date', 'nom_rwd_chosen', 'acc',
       'lrchoice', 'rt-1_a', 'rt-1_b', 'rt-1_c', 'rt-2_a', 'rt-2_b', 'rt-2_c',
       'rt-3_a', 'rt-3_b', 'rt-3_c', 'rt-4_a', 'rt-4_b', 'rt-4_c', 'rt-5_a',
       'rt-5_b', 'rt-5_c', 'samePath_t-1_a', 'samePath_t-1_b',
       'samePath_t-1_c', 'fromport', 'fromport2', 'same2', 'rt-1_left',
       'rt-2_left', 'rt-3_left', 'rt-4_left', 'rt-5_left', 'samePath_t-1_left',
       'leftLast'],
      dtype='object')

In [14]:
def create_sameValtChoiceDict(photrats):
    ratSameValtChoiceProbs = {r:{'pDiffRwdVsOmSame':[],'pDiffRwdVsOmAlt':[]}\
     for r in photrats.triframe.loc[:,"rat"].unique()}
    ratNs = {r:{'pDiffRwdVsOmSameRwd':[],'pDiffRwdVsOmAltRwd':[],\
                'pDiffRwdVsOmSameOm':[],'pDiffRwdVsOmAltOm':[]}\
                 for r in photrats.triframe.loc[:,"rat"].unique()}
    for rat in photrats.triframe.rat.unique():
        for s in photrats.triframe.loc[photrats.triframe.rat==rat,"session"].unique():
            dat = photrats.triframe.loc[(photrats.triframe.rat==rat)&\
                (photrats.triframe.session==s)&(photrats.triframe.leftLast==1),:]
            ratSameValtChoiceProbs[rat]['pDiffRwdVsOmSame'].append(\
               dat.loc[(dat["samePath_t-1_left"]==1)&(dat["rt-1_left"]==1),"lrchoice"].mean()\
               -dat.loc[(dat["samePath_t-1_left"]==1)&(dat["rt-1_left"]==0),"lrchoice"].mean())
            ratSameValtChoiceProbs[rat]['pDiffRwdVsOmAlt'].append(\
               dat.loc[(dat["samePath_t-1_left"]==0)&(dat["rt-1_left"]==1),"lrchoice"].mean()\
               -dat.loc[(dat["samePath_t-1_left"]==0)&(dat["rt-1_left"]==0),"lrchoice"].mean())
            ratNs[rat]['pDiffRwdVsOmSameRwd'].append(dat.loc[\
                (dat["samePath_t-1_left"]==1)&(dat["rt-1_left"]==1),].shape[0])
            ratNs[rat]['pDiffRwdVsOmSameOm'].append(dat.loc[\
                (dat["samePath_t-1_left"]==1)&(dat["rt-1_left"]==0),].shape[0])
            ratNs[rat]['pDiffRwdVsOmAltRwd'].append(dat.loc[\
                (dat["samePath_t-1_left"]==0)&(dat["rt-1_left"]==1),].shape[0])
            ratNs[rat]['pDiffRwdVsOmAltOm'].append(dat.loc[\
                (dat["samePath_t-1_left"]==0)&(dat["rt-1_left"]==0),].shape[0])
    return ratSameValtChoiceProbs,ratNs

In [15]:
#wsdf_all = createAndSaveWsdf4R(photrats)
#choiceSameValtDf = create_PortChoiceSameValtDf(wsdf_all)
ratSameValtChoiceProbs,ratNs = create_sameValtChoiceDict(photrats)

### test if same and alt are sig dif from each other

In [23]:
sameVals = [np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmSame"]) for rat in ratSameValtChoiceProbs.keys()]
altVals = [np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmAlt"]) for rat in ratSameValtChoiceProbs.keys()]

In [20]:
from scipy.stats import wilcoxon

In [24]:
wilcoxon(sameVals,altVals)

WilcoxonResult(statistic=16.0, pvalue=0.275390625)

In [22]:
rwdSameSum = 0
omSameSum = 0
rwdAltSum = 0
omAltSum = 0
for rat in ratNs.keys():
    rwdSameSum += np.sum(ratNs[rat]['pDiffRwdVsOmSameRwd'])
    omSameSum += np.sum(ratNs[rat]['pDiffRwdVsOmSameOm'])
    rwdAltSum += np.sum(ratNs[rat]['pDiffRwdVsOmAltRwd'])
    omAltSum += np.sum(ratNs[rat]['pDiffRwdVsOmAltOm'])

In [23]:
rwdAltSum

1799

In [54]:
ratNs

{'IM-1272': {'pDiffRwdVsOmSameRwd': [55, 82, 49, 46, 43, 24, 82, 66],
  'pDiffRwdVsOmAltRwd': [35, 18, 14, 32, 41, 14, 38, 20],
  'pDiffRwdVsOmSameOm': [70, 77, 21, 18, 31, 26, 47, 47],
  'pDiffRwdVsOmAltOm': [36, 19, 8, 30, 12, 12, 29, 20]},
 'IM-1273': {'pDiffRwdVsOmSameRwd': [23, 28, 21, 18, 11],
  'pDiffRwdVsOmAltRwd': [18, 13, 17, 9, 21],
  'pDiffRwdVsOmSameOm': [26, 24, 11, 11, 23],
  'pDiffRwdVsOmAltOm': [13, 14, 16, 12, 12]},
 'IM-1276': {'pDiffRwdVsOmSameRwd': [34,
   41,
   42,
   23,
   41,
   31,
   17,
   42,
   25,
   56,
   91],
  'pDiffRwdVsOmAltRwd': [27, 18, 47, 8, 28, 30, 13, 18, 27, 38, 27],
  'pDiffRwdVsOmSameOm': [34, 35, 33, 10, 29, 21, 29, 39, 26, 41, 60],
  'pDiffRwdVsOmAltOm': [9, 24, 37, 21, 15, 5, 13, 24, 21, 16, 29]},
 'IM-1291': {'pDiffRwdVsOmSameRwd': [61, 39, 41, 55, 34, 9, 19, 41, 45],
  'pDiffRwdVsOmAltRwd': [28, 22, 12, 24, 16, 11, 23, 29, 13],
  'pDiffRwdVsOmSameOm': [48, 31, 41, 36, 47, 13, 24, 41, 23],
  'pDiffRwdVsOmAltOm': [23, 16, 21, 32, 9, 5, 

In [24]:
from scipy.stats import wilcoxon

In [25]:
def plotChoosePortSameVAlt(photrats,ratSameValtChoiceProbs):
    fig = plt.figure(figsize=(4,5))
    plt.axhline(y=0,ls='--',color='k',lw=1)
    ratmeans_samePath = []
    ratmeans_altPath = []
    for rat in ratSameValtChoiceProbs:
        ratmeans_samePath.append(np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmSame"]))
        ratmeans_altPath.append(np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmAlt"]))
        plt.scatter(x=["Same\nPath", "Alternative\nPath"],y=[ratmeans_samePath[-1],
                  ratmeans_altPath[-1]],color='k',s=50,marker='D')
    sns.barplot(x=["Same\nPath", "Alternative\nPath"],y=[np.mean(ratmeans_samePath),
                     np.mean(ratmeans_altPath)],palette=["#ff7f0e","#2ca02c"],alpha=.5)
    plt.ylabel("$\Delta$ P(choose port)",fontsize='xx-large')
    plt.suptitle("Port choice increase\nfollowing reward - omission",fontsize='xx-large')
    plt.tight_layout()
    plt.xticks()
    sameSigRats = get_sigRats_fromMeanList(ratmeans_samePath,altString="greater")
    plot_sigMarkers(sameSigRats,-.1,yval=0.31)
    altSigRats = get_sigRats_fromMeanList(ratmeans_altPath,altString="greater")
    plot_sigMarkers(altSigRats,.95,yval=0.31)
    return fig

def get_sigRats_fromMeanList(meanList,altString="two-sided"):
    stat,pval = wilcoxon(meanList,alternative=altString)
    print(stat,pval)
    return pval<0.001,pval<0.01,pval<0.05

In [27]:
fig = plotChoosePortSameVAlt(photrats,ratSameValtChoiceProbs)
plt.ylim(-.3,.3)
plt.yticks(np.arange(-3,4)/10)

55.0 0.0009765625
52.0 0.0048828125


([<matplotlib.axis.YTick at 0x7fb17876d1c0>,
 [Text(0, -0.05, '−0.05'),
  Text(0, 0.0, '0.00'),
  Text(0, 0.05, '0.05'),
  Text(0, 0.10000000000000002, '0.10'),
  Text(0, 0.15000000000000002, '0.15'),
  Text(0, 0.2, '0.20'),
  Text(0, 0.25000000000000006, '0.25')])

In [28]:
fig.savefig(photrats.directory_prefix+"/deltaPchoosePortSameValt.pdf")

- plot same v alt path regression results

In [4]:
regCoefs_same = pd.read_csv(photrats.directory_prefix+"pChooseLeftSamePathCoefsByRat.csv")#"wslsMbVMfCoefsByRat.csv")
regSum_same = pd.read_csv(photrats.directory_prefix+"pChooseLeftSamePathSum.csv")#"wslsMbVMfSummary.csv")
regCoefs_alt = pd.read_csv(photrats.directory_prefix+"pChooseLeftAltPathCoefsByRat.csv")
regSum_alt = pd.read_csv(photrats.directory_prefix+"pChooseLeftAltPathSum.csv")

- *make sure to set negative and positive y limit to be equal*

In [10]:
regSum_same

,Unnamed: 0,Estimate,Std. Error,z value,Pr(>|z|)
0,(Intercept),-0.805987,0.154850,-5.204960,1.940380e-07
1,rt.1_left,0.319348,0.067491,4.731707,2.226396e-06
2,rt.2_left,0.159567,0.066618,2.395235,1.660971e-02
3,rt.3_left,0.122287,0.075344,1.623050,1.045787e-01
4,rt.4_left,0.278020,0.078266,3.552229,3.819823e-04
5,rt.5_left,0.145036,0.081891,1.771096,7.654473e-02


In [8]:
xvals = ["same\npath","alternative\npath"]
fig = plt.figure(figsize=(4.5,5))
plt.bar(xvals,[regSum_same.Estimate[1],regSum_alt.Estimate[1]],color=["#ff7f0e",'#2ca02c'],alpha=0.5)
for i in range(len(regCoefs_same.loc[:,"rt.1_left"])):
    plt.scatter(xvals[0],regCoefs_same.loc[:,"rt.1_left"][i],c="k",edgecolors='k',s=30,marker='D')
    plt.scatter(xvals[1],regCoefs_alt.loc[:,"rt.1_left"][i],c='k',edgecolors='k',s=30,marker='D')
plt.ylim(-0.49,.49)
plt.axhline(y=0,color="k",ls="--",lw=2)
plt.ylabel("Regression Beta")
plt.tight_layout()
fig.savefig(photrats.directory_prefix+"pChooseLeftSameValtReg_corAlign.pdf")

In [9]:
fig = plt.figure(figsize=(3.7,4))
xvals = ["Same\nPath","Alternative\nPath"]
#sns.boxplot(data=regCoefs.loc[:,["prev_rwd_taken","prev_rwd_alt"]],color='lightgrey')#,alpha=.5)
plt.bar([0,1],regSum.loc[1:2,"Estimate"].values,alpha=0.5,color=["#2ca02c","#ff7f0e"])
sns.stripplot(data=regCoefs.loc[:,["prev_rwd_taken","prev_rwd_alt"]],color='k',size=8,marker='D',alpha=1)
#plt.axhline(y=0,ls='--',color='k')
plt.xticks([0,1],xvals)
plotRegSigLevel(regSum,0,.45,1)
plotRegSigLevel(regSum,1,.45,2)
plt.ylim(-0.5,0.5)
plt.axhline(y=0,ls='--',color='k',lw=1)
plt.ylabel("Port Choice "+r"$\beta$",fontsize='xx-large')
plt.tight_layout()

NameError: name 'regSum' is not defined

In [80]:
fig.savefig(photrats.directory_prefix+"choosePortSameVAltCoefs.pdf")

- visualize individual rat sensitivities to costs and benefits

In [11]:
get_log_pchoos_v_costNben(photrats)
add_scaledVarsToRegDf(photrats)
photrats.regdf = photrats.regdf.loc[photrats.regdf.tri>25,]

In [39]:
plot_choiceSigmoidsByRelativeCostsAndBen(photrats)

- plot individual regression weights for costs and benefits for each rat

In [31]:
np.mean(regWeightsByRat,axis=0)[1:3]

relative p(R)        1.509342
relative distance   -6.033173
dtype: float64

In [12]:
def plot_choiceRegWeightsByRat_MF(photrats):
    regWeightsByRat = calc_choiceRegWeightsByRat(photrats)
    regWeightsByRat.loc[:,"rat"] = np.array(photrats.triframe.rat.unique())
    xvals = ["p(Reward)","Distance\ncost"]#["left bias","relative\np(reward)","relative\ndistance"]
    females = ["IM-1291","IM-1292","IM-1531","IM-1532","IM-1533"]
    fig = plt.figure(figsize=(3.55,6))
    plt.bar([0,1],np.mean(regWeightsByRat,axis=0)[1:],color='grey',alpha=.5)
    sns.stripplot(data=regWeightsByRat.loc[~regWeightsByRat.rat.isin(females),\
                                           ["relative p(R)","relative distance"]],color='k',size=10,marker='D',alpha=.9)
    sns.stripplot(data=regWeightsByRat.loc[regWeightsByRat.rat.isin(females),\
                                           ["relative p(R)","relative distance"]],color='deeppink',size=10,marker='D',alpha=.9)
    plt.axhline(y=0,ls='--',color='k',lw=1)
    plt.xticks([0,1],xvals,fontsize="large",fontstyle="italic")
    #plt.ylim(-10,2.2)
    plt.ylabel("Port Choice ß",fontsize='xx-large',fontweight='bold')
    plt.xlabel("Choice Feature",fontsize='xx-large',fontweight='bold')
    plt.tight_layout()
    fig.savefig(photrats.directory_prefix+"pChooseLregCoefsIndividualRats_1.pdf")

In [13]:
plot_choiceRegWeightsByRat(photrats)

Optimization terminated successfully.
         Current function value: 0.534518
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.547970
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.560840
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591582
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.575395
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.500732
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.536611
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.560975
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.630448
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.620402
  

- adding significance test to individual rat regression weight calculation

In [31]:
from scipy import stats
def run_sklearnLogRegWithPval(X,y):
    lm = LogReg(fit_intercept=True).fit(X,y)
    params = np.append(lm.intercept_,lm.coef_)
    predictions = lm.predict(X)
    new_X = np.append(np.ones((len(X),1)), X, axis=1)
    M_S_E = (sum((y-predictions)**2))/(len(new_X)-len(new_X[0]))
    v_b = M_S_E*(np.linalg.inv(np.dot(new_X.T,new_X)).diagonal())
    s_b = np.sqrt(v_b)
    t_b = params/ s_b
    p_val =[2*(1-stats.t.cdf(np.abs(i),(len(new_X)-len(new_X[0])))) for i in t_b]
    return np.concatenate(params),p_val

def run_smLogRegWithPval(X,y):
    X = np.hstack([np.ones(len(X)).reshape(-1,1),X])
    mod = sm.Logit(y, X).fit()
    coefs = mod.summary2().tables[1]['Coef.'].values
    pvals = mod.summary2().tables[1]["P>|z|"].values
    return coefs,pvals
    
def calc_choiceRegWeightsByRat(photrats):
    try:
        photrats.regdf
    except:
        print("choose left regDf not yet saved. Getting now...")
        get_log_pchoos_v_costNben(photrats)
        add_scaledVarsToRegDf(photrats)
    regWeights = np.zeros((len(photrats.triframe.rat.unique()),3))
    ratSigLevels = np.zeros((len(photrats.triframe.rat.unique()),3))
    for r in range(len(photrats.regdf.rat.unique())):
        regDf = photrats.regdf.loc[\
            (photrats.regdf.rat==photrats.regdf.rat.unique()[r])&(photrats.regdf.tri>25),\
            ["session","choose_L","rwdDifScaled","lengthDifScaled"]]
        regDf = regDf.loc[regDf.notnull().all(axis=1),:]
        y = regDf.loc[:,"choose_L"]
        X = regDf.loc[:,["rwdDifScaled","lengthDifScaled"]]
        betas,pvals = run_smLogRegWithPval(X,y)
        regWeights[r,:] = betas
        ratSigLevels[r,:] = pvals
        ratRegDf = pd.DataFrame(regWeights,columns=["intercept","relative p(R)","relative distance"])
        ratRegDf.loc[:,["intercept p value","relative p(R) p value","relative distance p value"]] = ratSigLevels
    return ratRegDf

#todo: add marker difference for sig vs ns
def plot_choiceRegWeightsByRat(photrats):
    regWeightsByRat = calc_choiceRegWeightsByRat(photrats)
    xvals = ["Left\nbias","Reward","Dist"]#["left bias","relative\np(reward)","relative\ndistance"]
    fig = plt.figure(figsize=(3.55,4))
    plt.bar([0,1,2],np.mean(regWeightsByRat,axis=0),color='grey',alpha=.5)
    sigpoints = (regWeightsByRat.values[:,3:]<0.05).astype(int)
    for i in range(len(regWeightsByRat)):
        #markers = 
        plt.scatter(regWeightsByRat.columns[:3],regWeightsByRat)
    sns.stripplot(data=regWeightsByRat,color='k',size=5,marker='D',alpha=.9)
    plt.axhline(y=0,ls='--',color='k',lw=1)
    plt.xticks([0,1,2],xvals,fontsize="large",fontstyle="italic")
    #plt.ylim(-10,2.2)
    plt.ylabel("Port Choice ß",fontsize='xx-large',fontweight='bold')
    plt.xlabel("Choice Feature",fontsize='xx-large',fontweight='bold')
    plt.tight_layout()
    fig.savefig(photrats.directory_prefix+"pChooseLregCoefsIndividualRats.pdf")


In [36]:
regWeightsByRat.values[:,3:]<0.05

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

- get and save dataframe for mixed-effects p(choose L) regression in R

In [33]:
photrats.triframe.drop(['level_0'],axis=1,inplace=True)

In [34]:
get_log_pchoos_v_costNben(photrats)
add_scaledVarsToRegDf(photrats)
photrats.regdf.to_csv(photrats.directory_prefix+"costVbenDf4reg.csv")

- load regression results from R

In [46]:
regCoefs = pd.read_csv(photrats.directory_prefix+"chooseLregCoefsByRat.csv")
regSum = pd.read_csv(photrats.directory_prefix+"chooseLregSummary.csv")

- plot regression results from R

In [50]:
regSum

,Unnamed: 0,Estimate,Std. Error,z value,Pr(>|z|)
0,(Intercept),2.214101,0.309209,7.160538,8.036102e-13
1,pRwdScaled,1.613729,0.158150,10.203759,1.907441e-24
2,ldifScaled,-6.803168,0.548637,-12.400121,2.609282e-35


In [53]:
fig = plt.figure(figsize=(3.5,4))
xvals = ["p(reward)","Distance"]
sns.barplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='darkgrey',alpha=.5)
sns.stripplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='k',size=8,marker='X',alpha=.9)
plt.axhline(y=0,ls='--',color='k')
plt.xticks([0,1],xvals,fontweight="bold")
plotRegSigLevel(regSum,0,2.0,1)
plotRegSigLevel(regSum,1,2.0,2)
plt.ylim(-10,2.2)
plt.ylabel("Path choice ß value",fontsize='xx-large',fontweight='bold')
plt.tight_layout()

In [54]:
fig.savefig(photrats.directory_prefix+"chooseL2ndHalfRegCoefs.pdf")

- plot change in port/path choice around block change

In [35]:
badInd = photrats.triframe.loc[(photrats.triframe.session==105)&(photrats.triframe.block==4)].index[0]
photrats.triframe.drop(badInd,axis=0,inplace=True)

In [40]:
high2lowProb,high2lowDist,low2highProb,low2highDist = get_sessionProbAndDistChangeInds(photrats)

In [41]:
lowsProb,highsProb,lowcountProb,highcountProb = calc_pChoosePortAtProbBlkTrans(photrats,high2lowProb,low2highProb)
fig = plot_portChoiceAtProbBlkTrans(photrats,lowsProb,highsProb,smoothWin=5,blineStart=5,legend_on=False)
plt.ylim(-.18,.18)

didn't work for session 3 block 4
didn't work for session 3 block 5


(-0.18, 0.18)

In [42]:
fig.savefig(photrats.directory_prefix+"pVisitPortVprobBlockChangeTri.pdf")

In [13]:
photrats.triframe.reset_index(inplace=True)

In [43]:
lowsBar,highsBar,lowcountBar,highcountBar = calc_pChoosePathAtBarBlkTrans(photrats,high2lowDist,low2highDist)
fig = plot_portChoiceAtBarBlkTrans(photrats,lowsBar,highsBar,smoothWin=5,blineStart=5,legend_on=False)

/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/timkrausz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [44]:
fig.savefig(photrats.directory_prefix+"pVisitPortVbarBlockChangeTri.pdf")